In [179]:
import requests
import pandas as pd
from flatten_json import flatten


def request_api(url, endpoint):
    response = requests.get(url + endpoint + '.json')

    return response.json()


def get_total_records(json_data):
    total_records = int(json_data['MRData']['total'])

    return total_records


def get_data_from_rounds(url, total):
    all_data = []

    for round_n in range(total+1):
        round_url = url + f'{round_n}/'
        data = request_api(round_url, 'results')
        
        for results_data in data['MRData']['RaceTable']['Races'][0]['Results']:
            race_info = {
                'raceName': data['MRData']['RaceTable']['Races'][0]['raceName'],
                'date': data['MRData']['RaceTable']['Races'][0]['date']
            }
            driver_info = results_data['Driver']
            constructor_info = results_data['Constructor']
            fastest_lap_info = results_data.get('FastestLap', {})
            
            selected_data = {
                'number': results_data['number'],
                'position': results_data['position'],
                'points': results_data['points'],
                'grid': results_data['grid'],
                'laps': results_data['laps'],
                'status': results_data['status'],
                'Driver_driverId': driver_info['driverId'],
                'Constructor_name': constructor_info['name'],
                'FastestLap_AverageSpeed_speed': fastest_lap_info.get('AverageSpeed', {}).get('speed')
            }
            selected_data.update(race_info)
            all_data.append(selected_data)

    return all_data

def create_df(data):
    df = pd.DataFrame(data)

    return df


def normalize_columns_names(col):
    new_column_name = ""
    for val in col:
        if val.isupper() and new_column_name and new_column_name[-1] != "_":
            new_column_name += "_"
        new_column_name += val

    return new_column_name.lower()

In [180]:
SEASON = 2021

api_url = f"https://ergast.com/api/f1/{SEASON}/"

In [181]:
circuits_data = request_api(api_url, 'circuits')

In [182]:
number_of_rounds_on_season = get_total_records(circuits_data)

In [183]:
all_data = get_data_from_rounds(api_url, number_of_rounds_on_season)

In [184]:
df = create_df(all_data)

In [185]:
df.columns = [normalize_columns_names(col) for col in df.columns]

In [186]:
df

,number,position,points,grid,laps,status,driver_driver_id,constructor_name,fastest_lap_average_speed_speed,race_name,date
0,44,1,25,2,56,Finished,hamilton,Mercedes,207.235,Bahrain Grand Prix,2021-03-28
1,33,2,18,1,56,Finished,max_verstappen,Red Bull,208.984,Bahrain Grand Prix,2021-03-28
2,77,3,16,3,56,Finished,bottas,Mercedes,211.566,Bahrain Grand Prix,2021-03-28
3,4,4,12,7,56,Finished,norris,McLaren,206.398,Bahrain Grand Prix,2021-03-28
4,11,5,10,0,56,Finished,perez,Red Bull,207.334,Bahrain Grand Prix,2021-03-28
...,...,...,...,...,...,...,...,...,...,...,...
435,5,16,0,17,44,Collision damage,vettel,Aston Martin,236.375,Saudi Arabian Grand Prix,2021-12-05
436,11,17,0,5,14,Collision,perez,Red Bull,236.104,Saudi Arabian Grand Prix,2021-12-05
437,9,18,0,20,14,Collision,mazepin,Haas F1 Team,229.036,Saudi Arabian Grand Prix,2021-12-05
438,63,19,0,14,14,Collision,russell,Williams,231.211,Saudi Arabian Grand Prix,2021-12-05


In [187]:
df.columns

Index(['number', 'position', 'points', 'grid', 'laps', 'status',
       'driver_driver_id', 'constructor_name',
       'fastest_lap_average_speed_speed', 'race_name', 'date'],
      dtype='object')

In [188]:
df.fillna(0)


,number,position,points,grid,laps,status,driver_driver_id,constructor_name,fastest_lap_average_speed_speed,race_name,date
0,44,1,25,2,56,Finished,hamilton,Mercedes,207.235,Bahrain Grand Prix,2021-03-28
1,33,2,18,1,56,Finished,max_verstappen,Red Bull,208.984,Bahrain Grand Prix,2021-03-28
2,77,3,16,3,56,Finished,bottas,Mercedes,211.566,Bahrain Grand Prix,2021-03-28
3,4,4,12,7,56,Finished,norris,McLaren,206.398,Bahrain Grand Prix,2021-03-28
4,11,5,10,0,56,Finished,perez,Red Bull,207.334,Bahrain Grand Prix,2021-03-28
...,...,...,...,...,...,...,...,...,...,...,...
435,5,16,0,17,44,Collision damage,vettel,Aston Martin,236.375,Saudi Arabian Grand Prix,2021-12-05
436,11,17,0,5,14,Collision,perez,Red Bull,236.104,Saudi Arabian Grand Prix,2021-12-05
437,9,18,0,20,14,Collision,mazepin,Haas F1 Team,229.036,Saudi Arabian Grand Prix,2021-12-05
438,63,19,0,14,14,Collision,russell,Williams,231.211,Saudi Arabian Grand Prix,2021-12-05


In [189]:
df_ops=df.drop_duplicates() 

In [190]:
df['number'] = df['number'].astype(int)
df['position'] = df['position'].astype(int)
df['points'] = df['points'].astype(float)
df['grid'] = df['grid'].astype(int)
df['laps'] = df['laps'].astype(int)
df['fastest_lap_average_speed_speed'] = df['fastest_lap_average_speed_speed'].astype(float)
df['date'] = pd.to_datetime(df['date'])

In [191]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   number                           440 non-null    int32         
 1   position                         440 non-null    int32         
 2   points                           440 non-null    float64       
 3   grid                             440 non-null    int32         
 4   laps                             440 non-null    int32         
 5   status                           440 non-null    object        
 6   driver_driver_id                 440 non-null    object        
 7   constructor_name                 440 non-null    object        
 8   fastest_lap_average_speed_speed  403 non-null    float64       
 9   race_name                        440 non-null    object        
 10  date                             440 non-null    datetime64[ns

In [192]:
df_ops.fillna(0)

,number,position,points,grid,laps,status,driver_driver_id,constructor_name,fastest_lap_average_speed_speed,race_name,date
0,44,1,25,2,56,Finished,hamilton,Mercedes,207.235,Bahrain Grand Prix,2021-03-28
1,33,2,18,1,56,Finished,max_verstappen,Red Bull,208.984,Bahrain Grand Prix,2021-03-28
2,77,3,16,3,56,Finished,bottas,Mercedes,211.566,Bahrain Grand Prix,2021-03-28
3,4,4,12,7,56,Finished,norris,McLaren,206.398,Bahrain Grand Prix,2021-03-28
4,11,5,10,0,56,Finished,perez,Red Bull,207.334,Bahrain Grand Prix,2021-03-28
...,...,...,...,...,...,...,...,...,...,...,...
435,5,16,0,17,44,Collision damage,vettel,Aston Martin,236.375,Saudi Arabian Grand Prix,2021-12-05
436,11,17,0,5,14,Collision,perez,Red Bull,236.104,Saudi Arabian Grand Prix,2021-12-05
437,9,18,0,20,14,Collision,mazepin,Haas F1 Team,229.036,Saudi Arabian Grand Prix,2021-12-05
438,63,19,0,14,14,Collision,russell,Williams,231.211,Saudi Arabian Grand Prix,2021-12-05


In [193]:
df_ops.isna().sum() 

number                              0
position                            0
points                              0
grid                                0
laps                                0
status                              0
driver_driver_id                    0
constructor_name                    0
fastest_lap_average_speed_speed    36
race_name                           0
date                                0
dtype: int64

In [194]:
csv_output_path = "dados_f1.csv"
df_ops.to_csv(csv_output_path, index=False)